# Feature attribution

In [ ]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2021"

## Contents

1. [Selectivity examples](#Selectivity-examples)
1. [Simple feed-forward classifier example](#Simple-feed-forward-classifier-example)
1. [Bag-of-words classifier for the SST](#Bag-of-words-classifier-for-the-SST)
1. [BERT example](#BERT-example)

## Overview

This notebook is an experimental extension of the CS224u course code. It focuses on the [Integrated Gradients](https://arxiv.org/abs/1703.01365) method for feature attribution, with comparisons to the "inputs $\times$ gradients" method. To run the notebook, first install [the Captum library](https://captum.ai/):

```pip install captum```

This is not currently a required installation (but it will be in future years).

## Selectivity examples

In [2]:
import numpy as np
import torch
import torch.nn as nn
from captum.attr import IntegratedGradients
from captum.attr import InputXGradient

In [3]:
class SelectivityAssessor(nn.Module):
    """Model used by Sundararajan et al, section 2.1 to show that
    input * gradients violates their selectivity axiom.
    """
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()

    def forward(self, X):
        return 1.0 - self.relu(1.0 - X)

In [4]:
sel_mod = SelectivityAssessor()

Simple inputs with just one feature:

In [5]:
X_sel = torch.FloatTensor([[0.0], [2.0]])

The outputs for our two examples differ:

In [6]:
sel_mod(X_sel)

tensor([[0.],
        [1.]])

However, `InputXGradient` assigns the same importance to the feature across the two examples, violating selectivity:

In [7]:
X_sel.requires_grad = True
sel_amod = InputXGradient(sel_mod)
sel_amod.attribute(X_sel)

tensor([[0.],
        [-0.]], grad_fn=<MulBackward0>)

Integrated gradients addresses the problem by averaging gradients across all interpolated representations between the baseline and the actual input:

In [8]:
ig_sel = IntegratedGradients(sel_mod)

In [9]:
sel_baseline = torch.FloatTensor([[0.0]])

In [10]:
ig_sel.attribute(X_sel, sel_baseline)

tensor([[0.],
        [1.]], dtype=torch.float64, grad_fn=<MulBackward0>)

A toy implementation to help bring out what is happening:

In [11]:
def ig_reference_implementation(model, x, base, m=50):
    vals = []
    for k in range(m):
        # Interpolated representation:
        xx = (base + (k/m)) * (x - base)
        # Gradient for the interpolated example:
        xx.requires_grad = True
        y = model(xx)
        (grads, ) = torch.autograd.grad(y.unbind(), xx)
        vals.append(grads)
    return (1 / m) * torch.cat(vals).sum() * (x - base)

In [12]:
ig_reference_implementation(sel_mod, torch.FloatTensor([[2.0]]), sel_baseline)

tensor([[1.]])

## Simple feed-forward classifier example

In [13]:
from captum.attr import IntegratedGradients
from sklearn.datasets import make_classification
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier

In [14]:
X_cls, y_cls = make_classification(
    n_samples=5000,
    n_classes=3,
    n_features=5,
    n_informative=3,
    n_redundant=0,
    random_state=42)

The classification problem has two uninformative features:

In [15]:
mutual_info_classif(X_cls, y_cls)

array([0.20138107, 0.02833358, 0.11584416, 0.        , 0.        ])

In [16]:
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(X_cls, y_cls)

In [17]:
classifier = TorchShallowNeuralClassifier()

In [18]:
_ = classifier.fit(X_cls_train, y_cls_train)

Stopping after epoch 364. Training loss did not improve more than tol=1e-05. Final error is 1.3800464868545532.

In [19]:
cls_preds = classifier.predict(X_cls_test)

In [20]:
accuracy_score(y_cls_test, cls_preds)

0.844

In [21]:
classifier_ig = IntegratedGradients(classifier.model)

In [22]:
classifier_baseline = torch.zeros(1, X_cls_train.shape[1])

Integrated gradients with respect to the actual labels:

In [23]:
classifier_attrs = classifier_ig.attribute(
    torch.FloatTensor(X_cls_test),
    classifier_baseline,
    target=torch.LongTensor(y_cls_test))

Average attribution is low for the two uninformative features:

In [24]:
classifier_attrs.mean(axis=0)

tensor([ 0.9169,  0.6473,  0.5929, -0.0190, -0.0079], dtype=torch.float64)

## Bag-of-words classifier for the SST

In [25]:
from collections import Counter
from captum.attr import IntegratedGradients
from nltk.corpus import stopwords
from operator import itemgetter
import os
from sklearn.metrics import classification_report
import torch
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
import sst

In [26]:
SST_HOME = os.path.join("data", "sentiment")

Bag-of-word featurization with stopword removal to make this a little easier to study:

In [27]:
stopwords = set(stopwords.words('english'))

def phi(text):
    return Counter([w for w in text.lower().split() if w not in stopwords])

In [28]:
def fit_mlp(X, y):
    mod = TorchShallowNeuralClassifier(early_stopping=True)
    mod.fit(X, y)
    return mod

In [29]:
experiment = sst.experiment(
    sst.train_reader(SST_HOME),
    phi,
    fit_mlp,
    sst.dev_reader(SST_HOME))

Stopping after epoch 43. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 0.5488652437925339

              precision    recall  f1-score   support

    negative      0.619     0.645     0.632       428
     neutral      0.276     0.162     0.204       229
    positive      0.616     0.723     0.665       444

    accuracy                          0.576      1101
   macro avg      0.504     0.510     0.500      1101
weighted avg      0.546     0.576     0.556      1101



Trained model:

In [30]:
sst_classifier = experiment['model']

Captum needs to have labels as indices rather than strings:

In [31]:
sst_classifier.classes_

['negative', 'neutral', 'positive']

In [32]:
y_sst_test = [sst_classifier.classes_.index(label)
              for label in experiment['assess_datasets'][0]['y']]

sst_preds = [sst_classifier.classes_.index(label)
             for label in experiment['predictions'][0]]

Our featurized test set:

In [33]:
X_sst_test = experiment['assess_datasets'][0]['X']

Feature names to help with analyses:

In [34]:
fnames = experiment['train_dataset']['vectorizer'].get_feature_names()

Integrated gradients:

In [35]:
sst_ig = IntegratedGradients(sst_classifier.model)

All-0s baseline:

In [36]:
sst_baseline = torch.zeros(1, experiment['train_dataset']['X'].shape[1])

Attributions with respect to the model's predictions:

In [37]:
sst_attrs = sst_ig.attribute(
    torch.FloatTensor(X_sst_test),
    sst_baseline,
    target=torch.LongTensor(sst_preds))

Helper functions for error analysis:

In [38]:
def error_analysis(gold=1, predicted=2):
    err_ind = [i for i, (g, p) in enumerate(zip(y_sst_test, sst_preds))
               if g == gold and p == predicted]
    attr_lookup = create_attr_lookup(sst_attrs[err_ind])
    return attr_lookup, err_ind

def create_attr_lookup(attrs):
    mu = attrs.mean(axis=0).detach().numpy()
    return sorted(zip(fnames, mu), key=itemgetter(1), reverse=True)

In [39]:
sst_attrs_lookup, sst_err_ind = error_analysis(gold=1, predicted=2)

In [40]:
sst_attrs_lookup[: 5]

[('fun', 0.07195141192484793),
 ('.', 0.058159633617987525),
 ('solid', 0.043577568037680446),
 ('film', 0.039300311169579966),
 ('makes', 0.0391925295271524)]

Error analysis for a specific example:

In [41]:
ex_ind = sst_err_ind[0]

In [42]:
experiment['assess_datasets'][0]['raw_examples'][ex_ind]

'No one goes unindicted here , which is probably for the best .'

In [43]:
ex_attr_lookup = create_attr_lookup(sst_attrs[ex_ind:ex_ind+1])

In [44]:
[(f, a) for f, a in ex_attr_lookup if a != 0]

[('best', 0.8184420312560889),
 ('.', 0.06119374157776072),
 ('probably', 0.058142852198823335),
 (',', 0.02635357016837504),
 ('one', -0.08374079113012509),
 ('goes', -0.19494963005042434)]

## BERT example

In [45]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz

In [46]:
hf_weights_name = 'cardiffnlp/twitter-roberta-base-sentiment'

In [47]:
hf_tokenizer = AutoTokenizer.from_pretrained(hf_weights_name)

In [48]:
hf_model = AutoModelForSequenceClassification.from_pretrained(hf_weights_name)

In [49]:
def hf_predict_one_proba(text):
    input_ids = hf_tokenizer.encode(
        text, add_special_tokens=True, return_tensors='pt')
    hf_model.eval()
    with torch.no_grad():
        logits = hf_model(input_ids)[0]
        preds = F.softmax(logits, dim=1)
    hf_model.train()
    return preds.squeeze(0)

In [50]:
def hf_ig_encodings(text):
    pad_id = hf_tokenizer.pad_token_id
    cls_id = hf_tokenizer.cls_token_id
    sep_id = hf_tokenizer.sep_token_id
    input_ids = hf_tokenizer.encode(text, add_special_tokens=False)
    base_ids = [pad_id] * len(input_ids)
    input_ids = [cls_id] + input_ids + [sep_id]
    base_ids = [cls_id] + base_ids + [sep_id]
    return torch.LongTensor([input_ids]), torch.LongTensor([base_ids])

In [51]:
def hf_ig_analyses(text2class):
    data = []
    for text, true_class in text2class.items():
        score_vis = hf_ig_analysis_one(text, true_class)
        data.append(score_vis)
    viz.visualize_text(data)


def hf_ig_analysis_one(text, true_class):
    # Option to look at different layers:
    # layer = model.roberta.encoder.layer[0]
    # layer = model.roberta.embeddings.word_embeddings
    layer = hf_model.roberta.embeddings

    def ig_forward(inputs):
        return hf_model(inputs).logits

    ig = LayerIntegratedGradients(ig_forward, layer)

    input_ids, base_ids = hf_ig_encodings(text)

    attrs, delta = ig.attribute(
        input_ids,
        base_ids,
        target=true_class,
        return_convergence_delta=True)

    # Summarize and z-score normalize the attributions
    # for each representation in `layer`:
    scores = attrs.sum(dim=-1).squeeze(0)
    scores = (scores - scores.mean()) / scores.norm()

    # Intuitive tokens to help with analysis:
    raw_input = hf_tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])
    # RoBERTa-specific clean-up:
    raw_input = [x.strip("Ġ") for x in raw_input]

    # Predictions for comparisons:
    pred_probs = hf_predict_one_proba(text)
    pred_class = pred_probs.argmax()

    score_vis = viz.VisualizationDataRecord(
        word_attributions=scores,
        pred_prob=pred_probs.max(),
        pred_class=pred_class,
        true_class=true_class,
        attr_class=None,
        attr_score=attrs.sum(),
        raw_input=raw_input,
        convergence_score=delta)

    return score_vis

In [52]:
score_vis = hf_ig_analyses({
    "They said it would be great, and they were right.": 2,
    "They said it would be great, and they were wrong.": 0,
    "They were right to say it would be great.": 2,
    "They were wrong to say it would be great.": 0,
    "They said it would be stellar, and they were correct.": 2,
    "They said it would be stellar, and they were incorrect.": 0})